# Extract Merchant ID and Product ID from link

In [108]:
import re

# Sample link
link = "https://www.trendyol.com/huawei/band-8-akilli-saat-sakura-pembesi-huawei-turkiye-garantili-p-736891275?boutiqueId=638145&merchantId=968"

# Regular expression patterns
product_id_pattern = r'\/([^\/]+)-p-(\d+)'
merchant_id_pattern = r'merchantId=(\d+)'

# Extracting product_id
product_id_match = re.search(product_id_pattern, link)
if product_id_match:
    product_id_text = product_id_match.group(1)
    product_id_number = product_id_match.group(2)
    print("Product ID Text:", product_id_text)
    print("Product ID Number:", product_id_number)

# Extracting merchant_id
merchant_id_match = re.search(merchant_id_pattern, link)
if merchant_id_match:
    merchant_id = merchant_id_match.group(1)
    print("Merchant ID Number:", merchant_id)

product_code = product_id_text + "-" +  product_id_number



Product ID Text: band-8-akilli-saat-sakura-pembesi-huawei-turkiye-garantili
Product ID Number: 736891275
Merchant ID Number: 968


In [111]:
product_id = product_id_text + "-p-" +  product_id_number
product_id

'band-8-akilli-saat-sakura-pembesi-huawei-turkiye-garantili-p-736891275'

# Scrapping Comments

In [112]:
import requests
import json

url = f"https://public-mdc.trendyol.com/discovery-web-socialgw-service/reviews/relax/{product_id}/yorumlar?merchantId={merchant_id}&sav=true&culture=tr-TR&storefrontId=1&RRsocialproofAbTesting=B&logged-in=true&isBuyer=false&channelId=1"
print(url)
r = requests.get(url)
content= r.json()
script_code = content["result"]["hydrateScript"]
# print(script_code)
index1 = script_code.index("__ =")
index2 = script_code.index("window.TYPageName")
# print(index1, index2, script_code[index1+5:index2-14])
data = json.loads(script_code[index1+5:index2-14])
comments = data["ratingAndReviewResponse"]["ratingAndReview"]["productReviews"]["content"]
for comment in comments:
    print("RATE:", comment["rate"])
    print("COMMENT:", comment["comment"])

https://public-mdc.trendyol.com/discovery-web-socialgw-service/reviews/relax/band-8-akilli-saat-sakura-pembesi-huawei-turkiye-garantili-p-736891275/yorumlar?merchantId=968&sav=true&culture=tr-TR&storefrontId=1&RRsocialproofAbTesting=B&logged-in=true&isBuyer=false&channelId=1
RATE: 5
COMMENT: fazla rahatsız etmeyin aşk yaşıyorum saatimle😅
RATE: 5
COMMENT: Hızlı ve düzgün geldi.  Eşime aldım. Çok beğendi ve memnun kaldı. Bir çok özelliği mevcut. Kullanımı kolay.
RATE: 5
COMMENT: kalite efendim
RATE: 5
COMMENT: İlk başta alisamamistim şimdi hayatımın bir parçası oldu severek kullanıyorum çok kaba değil çok küçük de değil aşırı şık diyemem ama çok ergonomik olduğunu düşünüyorum. alın bence
RATE: 5
COMMENT: Urun gayet hos naif kullanisli ozellikli kiz arkadasima aldim cok sevdi
RATE: 5
COMMENT: Hediye olarak sevgilime aldım. Çok beğendi. Eksiği yok. Çok başarılı bir saat
RATE: 5
COMMENT: kız arkadaşıma aldım begendi gayet güzel tavsiye ederim
RATE: 4
COMMENT: kardeşime aldım memnunn
RATE: 5

# Scrapping Product Description and Features

In [113]:
import requests
from bs4 import BeautifulSoup

url = f'https://www.trendyol.com/comedones/{product_id}?boutiqueId=61&merchantId={merchant_id}&sav=true'
response = requests.get(url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')
text_data = response.text

In [116]:
index1 = text_data.index("window.__PRODUCT_DETAIL_APP_INITIAL_STATE")
index2 = text_data.rindex("__PRODUCT_DETAIL_APP_INITIAL_STATE__.product.brand.name")
index1, index2

(143444, 217015)

In [118]:
json_string = text_data[index1+44:index2 - 1234]
data = json.loads(json_string)

In [119]:
data["product"]["descriptions"]

[{'text': 'Bu ürün\xa0[page="merchant_info"]Trendyol[/page]\xa0tarafından gönderilecektir.',
  'priority': 2},
 {'text': 'Kampanya fiyatından satılmak üzere 100 adetten fazla stok sunulmuştur.',
  'priority': 4},
 {'text': 'Ürüne ait garanti belgesi ile tanıtma ve kullanım kılavuzları; kağıt ortamında paylaşılabileceği gibi elektronik ortamda erişim sağlayabileceğiniz, link ve karekod biçiminde de sunulabilecektir. Ürün teslimi sonrası ambalajın, paketin, kutunun dikkatle açılması tavsiye edilir.',
  'priority': 18},
 {'text': 'Bir ürün, birden fazla satıcı tarafından satılabilir. Birden fazla satıcı tarafından satışa sunulan ürünlerin satıcıları ürün için belirledikleri fiyata, satıcı puanlarına, teslimat statülerine, ürünlerdeki promosyonlara, kargonun bedava olup olmamasına ve ürünlerin hızlı teslimat ile teslim edilip edilememesine, ürünlerin stok ve kategorileri bilgilerine göre sıralanmaktadır.',
  'priority': 1001},
 {'text': 'Bu üründen en fazla\xa01 adet sipariş verilebilir.\x

In [120]:
data["product"]["attributes"]

[{'key': {'name': 'Garanti Tipi', 'id': 290},
  'value': {'name': 'Huawei Türkiye Garantili', 'id': 286816},
  'starred': False,
  'description': '',
  'mediaUrls': []},
 {'key': {'name': 'İşletim Tipi', 'id': 170},
  'value': {'name': 'Android', 'id': 1957},
  'starred': False,
  'description': '',
  'mediaUrls': []},
 {'key': {'name': 'Renk', 'id': 348},
  'value': {'name': 'Pembe', 'id': 12},
  'starred': False,
  'description': '',
  'mediaUrls': []},
 {'key': {'name': 'Garanti Süresi', 'id': 433},
  'value': {'name': '2 Yıl', 'id': 20637},
  'starred': False,
  'description': '',
  'mediaUrls': []},
 {'key': {'name': 'Adımsayar', 'id': 484},
  'value': {'name': 'Var', 'id': 23607},
  'starred': False,
  'description': '',
  'mediaUrls': []},
 {'key': {'name': 'GPS', 'id': 692},
  'value': {'name': 'Var', 'id': 255877},
  'starred': False,
  'description': '',
  'mediaUrls': []},
 {'key': {'name': 'Elektriksel Kalp Sensörü', 'id': 861},
  'value': {'name': 'Var', 'id': 791579},
  '